In [1]:
import hls4ml
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects
import tensorflow as tf 
from util import create_config

model_dir  = './models/deeppicar-stats/models/1100-000_64x64x1_0.1/'
model_name = f'{model_dir}/0-1100-000-0.1.h5'

co = {}
_add_supported_quantized_objects(co)
model = load_model(model_name, custom_objects=co)

2025-04-28 20:07:27.704901: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-28 20:07:27.705994: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 20:07:27.726757: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-28 20:07:27.727155: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-28 20:07:28.057541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

## Print Model Summary

In [2]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (QConv2D)             (None, 30, 30, 2)         52        
                                                                 
 q_activation (QActivation)  (None, 30, 30, 2)         0         
                                                                 
 q_conv2d (QConv2D)          (None, 13, 13, 4)         204       
                                                                 
 q_activation_1 (QActivation  (None, 13, 13, 4)        0         
 )                                                               
                                                                 
 q_conv2d_1 (QConv2D)        (None, 5, 5, 5)           505       
                                                                 
 q_activation_2 (QActivation  (None, 5, 5, 5)          0         
 )                                                      

In [3]:
### Create HLS Model
directory = model_dir + '/hls/project_1'
config, hls_model = create_config(model, output_dir=directory)
print(config)

Interpreting Sequential
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 64, 64, 1]], output shape: [None, 64, 64, 1]
Layer name: conv1, layer type: QConv2D, input shapes: [[None, 64, 64, 1]], output shape: [None, 30, 30, 2]
Layer name: q_activation, layer type: Activation, input shapes: [[None, 30, 30, 2]], output shape: [None, 30, 30, 2]
Layer name: q_conv2d, layer type: QConv2D, input shapes: [[None, 30, 30, 2]], output shape: [None, 13, 13, 4]
Layer name: q_activation_1, layer type: Activation, input shapes: [[None, 13, 13, 4]], output shape: [None, 13, 13, 4]
Layer name: q_conv2d_1, layer type: QConv2D, input shapes: [[None, 13, 13, 4]], output shape: [None, 5, 5, 5]
Layer name: q_activation_2, layer type: Activation, input shapes: [[None, 5, 5, 5]], output shape: [None, 5, 5, 5]
Layer name: flatten, layer type: Reshape, input shapes: [[None, 5, 5, 5]], output shape: [None, 125]
Layer name: output_1, layer type: QDense, input shapes: [[None, 125]], outp

In [4]:
hls_model.compile()

Writing HLS project
Done


In [18]:
### Check to see if it has the similar error
from get_dataset import get_dataset
import numpy as np

imgs_train, imgs_test, vals_train, vals_test = get_dataset('./deeppicar')

preds = hls_model.predict(imgs_test.astype(np.float32))
vals_test = np.array(vals_test)

# mse
print(f'HLS MSE: {np.mean(((preds.reshape(-1) - vals_test.reshape(-1)) ** 2))}')

preds = model.predict(imgs_test.astype(np.float32))
vals_test = np.array(vals_test)

# mse
print(f'QKeras MSE: {np.mean(((preds.reshape(-1) - vals_test.reshape(-1)) ** 2))}')

HLS MSE: 0.05487402528524399
79/79 [==============================] - 0s 1ms/step
QKeras MSE: 0.056417979300022125


In [19]:
hls_model.build(csim=False)


****** Vitis HLS - High-Level Synthesis from C, C++ and OpenCL v2024.2 (64-bit)
  **** SW Build 5238294 on Nov  8 2024
  **** IP Build 5239520 on Sun Nov 10 16:12:51 MST 2024
  **** SharedData Build 5239561 on Fri Nov 08 14:39:27 MST 2024
  **** Start of session at: Mon Apr 28 20:12:45 2025
    ** Copyright 1986-2022 Xilinx, Inc. All Rights Reserved.
    ** Copyright 2022-2024 Advanced Micro Devices, Inc. All Rights Reserved.

source /opt/Xilinx/Vitis/2024.2/scripts/vitis_hls/hls.tcl -notrace
INFO: [HLS 200-10] For user 'petertso' on host 'pop-os' (Linux_x86_64 version 6.9.3-76060903-generic) on Mon Apr 28 20:12:46 CDT 2025
INFO: [HLS 200-10] On os Pop!_OS 22.04 LTS
INFO: [HLS 200-10] In directory '/home/petertso/Documents/deeppicarmicro/NAS/All/models/deeppicar-stats/models/1100-000_64x64x1_0.1/hls/project_1'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-1510] Running: open_project myproject_prj 
INFO: [HLS 200-10] Creating and opening project '/home/petertso/Documents/deeppicar

{'CSynthesisReport': {'TargetClockPeriod': '30.00',
  'EstimatedClockPeriod': '20.557',
  'BestLatency': '225239',
  'WorstLatency': '225289',
  'IntervalMin': '16386',
  'IntervalMax': '225282',
  'BRAM_18K': '22',
  'DSP': '13',
  'FF': '9470',
  'LUT': '14109',
  'URAM': '0',
  'AvailableBRAM_18K': '100',
  'AvailableDSP': '90',
  'AvailableFF': '41600',
  'AvailableLUT': '20800',
  'AvailableURAM': '0'}}

In [20]:
hls4ml.report.read_vivado_report(directory)

Found 1 solution(s) in ./models/deeppicar-stats/models/1100-000_64x64x1_0.1//hls/project_1/myproject_prj.
Reports for solution "solution1":

C simulation report not found.
SYNTHESIS REPORT:
== Vitis HLS Report for 'myproject'
* Date:           Mon Apr 28 20:13:02 2025

* Version:        2024.2 (Build 5238294 on Nov  8 2024)
* Project:        myproject_prj
* Solution:       solution1 (Vivado IP Flow Target)
* Product family: artix7
* Target device:  xc7a35t-cpg236-1


== Performance Estimates
+ Timing: 
    * Summary: 
    +--------+----------+-----------+------------+
    |  Clock |  Target  | Estimated | Uncertainty|
    +--------+----------+-----------+------------+
    |ap_clk  |  30.00 ns|  20.557 ns|     8.10 ns|
    +--------+----------+-----------+------------+

+ Latency: 
    * Summary: 
    +---------+---------+----------+----------+-------+--------+----------+
    |  Latency (cycles) |  Latency (absolute) |    Interval    | Pipeline |
    |   min   |   max   |    min   |    